In [1]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import re

# === CONFIGURACIÓN ===
CARPETA_EPUBS = "libros"   # carpeta donde guardas tus libros
AUTORES = {
    "Isabel Allende": ["Cuentos_de_Eva_Luna_Isabel_Allende.epub", "La_casa_de_los_espiritus_Isabel_Allende.epub"],
    "Carlos Ruiz Zafón": ["La_Sombra_del_Viento_Carlos_Ruiz_Zafon.epub", "El_Juego_del_Angel_Carlos_Ruiz_Zafon.epub"],
    "Eloy Moreno": ["El_regalo_Eloy_Moreno.epub", "Invisible_Eloy_Moreno.epub", "Tierra_Eloy_Moreno.epub", "El_boligrafo_de_gel_verde_Eloy_Moreno.epub"],
    "María Dueñas": ["Las_hijas_del_Capitan_Maria_Duenas_Vinuesa.epub", "El_tiempo_entre_costuras_Maria_Duenas_Vinuesa.epub"],
    "Santiago Posteguillo": ["Los_asesinos_del_emperador_Santiago_Posteguillo.epub", "Roma_soy_yo_Santiago_Posteguillo.epub"],
    "Javier Castillo": ["El_cuco_de_cristal_Javier_Castillo.epub", "El_dia_que_se_perdio_la_cordura_Javier_Castillo.epub", "La_grieta_del_silencio_Javier_Castillo.epub"],
}
PALABRAS_POR_FRAGMENTO = 300

# Cuántos fragmentos iniciales eliminar por libro
FRAGMENTOS_A_OMITIR_POR_LIBRO = 10

def extraer_texto_epub(path_epub):
    """Extrae texto limpio de un archivo epub"""
    libro = epub.read_epub(path_epub)
    textos = []
    for item in libro.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), "html.parser")
            texto = soup.get_text(separator=" ", strip=True)
            textos.append(texto)
    texto_total = " ".join(textos)
    # Limpieza básica
    texto_total = re.sub(r'\s+', ' ', texto_total)
    texto_total = re.sub(r'Capítulo\s+\d+', '', texto_total, flags=re.IGNORECASE)
    return texto_total.strip()

def fragmentar_texto(texto, palabras_por_fragmento=300):
    palabras = texto.split()
    fragmentos = []
    for i in range(0, len(palabras), palabras_por_fragmento):
        frag = palabras[i:i+palabras_por_fragmento]
        if len(frag) > palabras_por_fragmento // 2:
            fragmentos.append(" ".join(frag))
    return fragmentos

# === PROCESAR TODOS LOS LIBROS ===
data = []

for autor, libros in tqdm(AUTORES.items()):
    for libro in libros:
        ruta = Path(CARPETA_EPUBS) / libro
        texto = extraer_texto_epub(ruta)
        fragmentos = fragmentar_texto(texto, PALABRAS_POR_FRAGMENTO)
        # omitir los primeros N fragmentos de cada libro
        fragmentos = fragmentos[FRAGMENTOS_A_OMITIR_POR_LIBRO:]
        for frag in fragmentos:
            data.append({"autor": autor, "fragmento": frag})

# Construir DataFrame
df = pd.DataFrame(data)
# Filtro opcional por longitud mínima (mantener si lo deseas)
df = df[df['fragmento'].str.split().str.len() > 100]

# Balancear por autor: igualar al mínimo número de fragmentos
conteos = df.groupby('autor').size()
min_conteo = int(conteos.min()) if len(conteos) else 0
if min_conteo > 0:
    df = df.groupby('autor', group_keys=False).apply(
        lambda g: g.sample(min_conteo, random_state=42)
    )

# Guardar
df.to_csv("dataset_libros_extra.csv", index=False, encoding="utf-8")
print(" Dataset guardado como dataset_libros.csv")

100%|██████████| 6/6 [00:08<00:00,  1.37s/it]
C:\Users\marko\AppData\Local\Temp\ipykernel_2156\893627369.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('autor', group_keys=False).apply(


 Dataset guardado como dataset_libros.csv


In [2]:
df = pd.read_csv("dataset_libros_extra.csv", encoding="utf-8")
print(df['autor'].value_counts())

autor
Carlos Ruiz Zafón       814
Eloy Moreno             814
Isabel Allende          814
Javier Castillo         814
María Dueñas            814
Santiago Posteguillo    814
Name: count, dtype: int64
